# Interactive Molecular Dynamics with OpenMM, ASE and NanoVer

In this notebook, we run a pre-prepared OpenMM simulation of methane molecule and a carbon nanotube using ASE, serving it for interactive molecular dynamics using NanoVer.

We'll then connect NanoVer's NGLView client to it and apply some interactive forces directly from the notebook

In [1]:
from nanover.omni import OmniRunner
from nanover.ase import ASESimulation
from nanover.ase.omm_calculator import OpenMMCalculator

## Run the NanoVer server

First, we need to create an `ASESimulation` for our methane and nanotube system.

Let's do so by loading in a pre-prepared NanoVer OpenMM XML file, which contains:
- the PDB file
- the OpenMM XML file, containing:
  - the System
  - the Integrator

This format ensures that the whole simulation can be specified in a single file.

We then need to convert the OpenMM simulation loaded from this pre-prepared input file to an ASE `dynamics` object that uses our custom `OpenMMCalculator`, which can then be used to perform an ASE simulation that uses OpenMM to calculate the forces that propagate the dynamics.

The `ASESimulation` class takes a `dynamics` object constructed this way, and creates a simulation object that can then be served for interactive molecular dynamics using OpenMM via ASE. For an example of a more advanced setup, see [our neuraminidase notebook](./ase_openmm_neuraminidase.ipynb).

In [2]:
from ase.md import Langevin
import ase.units as ase_units
from nanover.openmm.serializer import deserialize_simulation

# Load the preprepared OpenMM input file
input_xml = "openmm_files/nanotube.xml"
with open(input_xml, "r") as infile:
    omm_simulation = deserialize_simulation(infile)

# Define the OpenMM calculator from the simulation
calculator = OpenMMCalculator(omm_simulation)

# Define the atoms object and set it's calculator as the OpenMMCalculator
atoms = calculator.generate_atoms()
atoms.calc = calculator

# Define the integrator for the simulation
dynamics = Langevin(atoms, timestep=1.0 * ase_units.fs, temperature_K=300, friction=1.0e-03)

nanotube_simulation = ASESimulation.from_ase_dynamics(dynamics, ase_atoms_to_frame_data=calculator.make_frame_converter())

Now we have our simulation ready, we can set up our NanoVer server to run the simulation. We run interactive ASE-OpenMM simulations in NanoVer using the `OmniRunner` class, which is in charge of serving and running the simulation.

In [3]:
# This will just automatically close previous instances of the runner if you re-run this cell multiple times
try:
    imd_runner.close()
except NameError:
    pass
imd_runner = OmniRunner.with_basic_server(nanotube_simulation, name="nanotube-ase-omm-server")
imd_runner.next()
imd_runner.pause()

In [4]:
imd_runner.print_basic_info()

Serving "nanotube-ase-omm-server" (ws://localhost:38801), discoverable on all interfaces on port 54545
Available simulations:
0: "Unnamed ASE OpenMM Simulation"


As we are running the OpenMM simulation via ASE, we can use the ASE functionality to run a few simulation steps to check that everything is working:

In [5]:
nanotube_simulation.dynamics.run(10)

True

In [6]:
nanotube_simulation.dynamics.get_number_of_steps()

15

In [7]:
# Note that this is the energy in eV, not kJ mol-1 (the standard units of ASE are different to those of OpenMM and NanoVer)
nanotube_simulation.dynamics.atoms.get_potential_energy()

7.963334667807109e+24

Now, let's leave the dynamics running dynamics in a background thread

In [8]:
imd_runner.play()

In [9]:
nanotube_simulation.dynamics.get_time()

1.4734042182696094

The server is running, so you can connect to it in VR and you'll see something like this:

![nanotube](./images/nanover_nanotube.png)



# Visualizing the simulation with NGLView 

One type of python client provided by NanoVer is the `NGLClient`, which enables visualisation of the trajectory of the molecular system with [NGLView](https://github.com/arose/nglview).

Below, we use this client to connect to the server to visualise the methane & nanotube system.

**Note**: Your browser needs to support WebGL (try Firefox or Edge)

In [10]:
from nanover.nglview import NGLClient

In [13]:
client = NGLClient.from_runner(imd_runner)

In [15]:
client.wait_until_basic_topology()
client.view

FETCHING VIEW


NGLWidget(n_components=2)

## Apply an Interactive Force 

Generally, you probably want to apply forces from the VR app, NanoVer iMD-VR - but you can do it from python! 

First, let's apply a couple of forces to hold to nanotube in place (there are better ways to do this) 

In [ ]:
from nanover.imd.particle_interaction import ParticleInteraction
import numpy as np

First, we ask the server for an ID for our interactions

In [ ]:
anchor1_id = client.start_interaction()

In [ ]:
anchor2_id = client.start_interaction()

Now, we can create two interactions, one at each end of the nanotube, and assign a position. 
You can repeatedly run the cell below, changing the positions or strength of the interactions. See if you can catch the methane! 

**Disclaimer**: This is *much* easier in VR.

In [ ]:
first_position = np.array((10,1,0)) # nanometers!
second_position = np.array((10,1,0))
anchor1 = ParticleInteraction(position=first_position, scale=10, particles=(0,), interaction_type='spring')
anchor2 = ParticleInteraction(position=second_position, scale=10, particles=(59,), interaction_type='spring')
client.update_interaction(anchor1_id, anchor1)
client.update_interaction(anchor2_id, anchor2)

In [ ]:
client.stop_interaction(anchor1_id)

In [ ]:
client.stop_interaction(anchor2_id)

As always, we should clean up after ourselves:

# Close the Client and Server

In [ ]:
client.close()
imd_runner.close()

# Next Steps

* The [NGLViewer notebook](nanover_nglview.ipynb) is a smaller notebook designed for visualizing your own simulations.
* Set up an OpenMM simulation with an AMBER file for a [protein-ligand system](ase_openmm_neuraminidase.ipynb) and simulate it in NanoVer
* Set up a simulation of a [graphene sheet](ase_openmm_graphene.ipynb) with parameters than can be controlled from the jupyter notebook.
* Visualize an [LSD receptor in a membrane](../mdanalysis/mdanalysis_lsd.ipynb) structure with MDAnalysis and NanoVer 